In [1]:
import json
import os
import shutil
import requests
import wget
import pickle

from pathlib import Path

In [2]:
class Post:
    def __init__(self, json_id,media):
        self. user_id        = ''
        self.timestamp       = ''
        self.user_name       = ''
        self.likes           = 0
        self.caption_picture = ''
        self.comments        = []
        self.json_id         = json_id
        self.media           = media

In [3]:
def show(post_list):
    for item in post_list:
        print("Object:",item,"\n")
        print("JSON:",item.json_id)
        print("Pic :",item.media)
        print("UserID:", item.user_id)
        print("Timestamp:", item.timestamp)
        print("Username:", item.user_name)
        print("Likes:", item.likes)
        print("Caption:", item.caption_picture,"\n\n")
        print("Comments", item.comments, "\n\n")

In [4]:
def read_filename():
    json_list = []
    
    media_list =[]

    for filename in os.listdir(os.getcwd()):
        if ".json" in filename and not "comments" in filename:
            json_list.append(filename)                   
        if ".jpg" in filename or ".mp4" in filename:
            media_list.append(filename)
                
    return json_list, media_list

In [5]:
def filename_to_object(json_list, media_list):
    split_at = 19
    post_list = []

    for item_j in json_list:
        media_object = []
        comments = ''
        txt= ''
        time = item_j[:split_at]
    
        for item_m in media_list:
            if time == item_m[:split_at]:
                media_object.append(item_m)
            else:
                pass
        
        
        post_list.append(Post(item_j, media_object))
        
    return post_list

In [14]:
def save_as_directory (post_list):
  
    split_at = 19
    directory = Path.cwd()/'sorted_Data'
    source = Path.cwd()    
    
    try:
        directory.mkdir()
    except FileExistsError:
        print("Sorted_Data-Ordner schon vorhanden")
        
    for i in range(len(post_list)):
        foldername = post_list[i].json_id[:split_at]
        folder_directory = directory/foldername
    
        f= open(post_list[i].json_id, 'r')
        json_dict = json.load(f)
        f.close()
        
        url = json_dict['node']['display_resources'][2]['src']
        pic = requests.get(url)
        
        if pic.status_code == 200:                  
            try:
                folder_directory.mkdir()
            except FileExistsError:
                print("Ordner", foldername, "schon vorhanden")
        
            post_list[i].commentstxt = post_list[i].json_id[:split_at]+'comments.txt'
            post_list[i].captiontxt  = post_list[i].json_id[:split_at]+'caption.txt'
            file_2 = folder_directory/post_list[i].commentstxt
            file_3 = folder_directory/post_list[i].captiontxt
            
            f_2 = open(file_2, 'w+', encoding = 'utf8')
            for line in post_list[i].comments:
                f_2.write(str(line)+'\n')
            f_2.close()
            
            f_3 = open(file_3, 'w+', encoding = 'utf8')
            for line in post_list[i].caption_picture:
                f_3.write(line)
            f_3.close()
        
            temp_1 = post_list[i].json_id+'.pkl'
            file_4 = folder_directory/temp_1
            with open(file_4, 'wb') as output:
                pickle.dump(post_list[i], output, pickle.HIGHEST_PROTOCOL)
            
            shutil.copyfile(source/post_list[i].json_id,folder_directory/post_list[i].json_id)    
            
            try:               
                for n in range(len(post_list[i].media)):
                    try:
                        url_2 = json_dict['node']['edge_sidecar_to_children']['edges'][n]['node']['display_resources'][2]['src']
                        pic_2 = requests.get(url_2)
                        open(folder_directory/post_list[i].media[n], 'wb').write(pic_2.content)
                    except KeyError:  
                        open(folder_directory/post_list[i].media[0], 'wb').write(pic.content)
                    except IndexError:
                        pass
            except FileExistsError:
                pass
        else:
            pass
  

In [1]:
def json_to_object (post_list):
    
    for i in range(len(post_list)):
        f = open(post_list[i].json_id, 'r')
        json_dict = json.load(f)
        f.close()    
        post_list[i].user_id = json_dict["node"]["owner"]['id']
        post_list[i].timestamp = json_dict["node"]["taken_at_timestamp"]
        post_list[i].user_name = json_dict["node"]["owner"]['username']
        post_list[i].likes = json_dict["node"]["edge_liked_by"]["count"]
        post_list[i].caption_picture = json_dict['node']["edge_media_to_caption"]['edges'][0]['node']['text']
        
        level_1 = len(json_dict['node']['edge_media_to_parent_comment']['edges'])
        comments = []
        flag = []

        for l in range(0,level_1):
    
            comments.append(json_dict['node']['edge_media_to_parent_comment']['edges'][l]['node']['text'])
            flag.append(1)
            level_2 = len(json_dict['node']['edge_media_to_parent_comment']['edges'][l]['node']['edge_threaded_comments']['edges'])
            
            for n in range(0,level_2):
        
                comments.append(json_dict['node']['edge_media_to_parent_comment']['edges'][l]['node']
                  ['edge_threaded_comments']['edges'][n]['node']['text'])
                flag.append(2)
       
        post_list[i].comments = list(zip(comments,flag))
       
    return post_list
        

In [ ]:
def read_data():
    post_list = []
    directory = Path.cwd()/'sorted_Data'
    folder_list = os.listdir(directory)
    
    for folder in folder_list:
        filename = folder+"_UTC.json.pkl"
        with open(directory/folder/filename, 'rb+') as openfile:
            temp = pickle.load(openfile)
            post_list.append(temp) 
            
    return(post_list)

In [19]:
def run():
    a,b = read_filename()
    posts = filename_to_object(a,b)
    posts_2 = json_to_object(posts)
    save_as_directory(posts_2)
       
    return posts_2    